## 函数
**函数可以赋予变量：**

In [2]:
def func(message):
    print(f"the message is {message}")
send_message=func
send_message("hello,world!")

the message is hello,world!


**函数可以当作参数，传入另一个函数中**

In [3]:
def get_message(message):
    return "got a message: "+message
def rot_call(func,message):
    print(func(message))
rot_call(get_message,"hello,world")

got a message: hello,world


**函数中定义函数，实现函数嵌套**

In [4]:
def func(message):
    def get_message(message):
        print("got a message:{}".format(message))
    return get_message(message)
func("hello,world")

got a message:hello,world


**函数实现闭包**

In [7]:
def func():
    def get_message(message):
        print(f"got a message:{message}")
    return get_message
send_message=func()
send_message("hello,world")

got a message:hello,world


In [9]:
def func():
    def get_message(message):
        print(f"got a message:{message}")
    return get_message
func()("hello,world")


got a message:hello,world


## 装饰器
Decorators is to modify the behavior of the function through a wrapper so we don’t have to actually modify the function.<br/>**application:Authentication, logging, input rationality checking, and caching**

### 基础装饰器

In [12]:
#common
def my_decorator(func):  
    def wrapper():
        print("wrapper of decorator")
        func()
    return wrapper
def greet():
    print("hello,world")
greet=my_decorator(greet)#greet指向内部函数wrapper(),wrapper()调用函数greet()
greet()

wrapper of decorator
hello,world


**@语法糖**
程序中有其他的函数需要做装饰，只需要在上方加上@装饰函数的名称

In [13]:
#unusual
def my_decorator(func):
    def wrapper():
        print("wrapper of decorator")
        func()
    return wrapper
@my_decorator   #greet=my_decorator(greet)
def greet():
    print("hello,world")
greet()

wrapper of decorator
hello,world


### 带有参数的装饰器

**单个参数**

In [14]:
def my_decorator(func):
    def wrapper(message):
        print('wrapper of decorator')
        func(message)
    return wrapper
@my_decorator  #greet=my_decorator(greet)
def greet(message):
    print(message)
greet('hello world')

wrapper of decorator
hello world


**多个参数**

In [32]:
def my_decorator(func): #接收任意类型的传参
    def wrapper(*args,**kwargs):
        print('wrapper of decorator')
        func(*args,**kwargs)
    return wrapper
@my_decorator  #greet=my_decorator(greet)
def greet(*args,**kwargs):
    print(*args,**kwargs)
greet(['hello world! ',"This is a variable parameter!"])

wrapper of decorator
['hello world! ', 'This is a variable parameter!']


In [28]:
def my_decorator(func):
    def wrapper(a=None,**kargs):
        print('wrapper of decorator')
        func(**kargs)
    return wrapper
@my_decorator  #greet=my_decorator(greet)
def greet(**kargs):
    print(**kargs)
greet(a="s")

wrapper of decorator



### 带有自定义参数的装饰器

In [40]:
def repeat(num):
    def my_decorator(func):
        def wrapper(*args,**kwargs):
            for i in range(num):
                print("wrapper of decorator")
                func(*args,**kwargs)
        return wrapper
    return my_decorator
@repeat(3)  #相当于greet=repeat(4)(greet)   带自定义参数，
def greet(message):
    print(message)
greet("hello,world")
print(greet.__name__) #元信息表示，greet作用域为wrapper()
help(greet)

wrapper of decorator
hello,world
wrapper of decorator
hello,world
wrapper of decorator
hello,world
wrapper
Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



In [60]:
from functools import wraps
def repeat(num):
    def my_decorator(func):
        @wraps(func)
        def wrapper(*args,**kwargs):
            for i in range(num):
                print("wrapper of decorator")
                func(*args,**kwargs)
        return wrapper
    return my_decorator
def greet(message):
    print(message)
greet=repeat(4)(greet)
print(greet.__name__)
greet("hello,world")

greet
wrapper of decorator
hello,world
wrapper of decorator
hello,world
wrapper of decorator
hello,world
wrapper of decorator
hello,world


**在编写装饰器时需要复制元信息。装饰器作用在某个函数上，通过复制，注解底层包装函数的重要的元信息比如名字、文档字符串、注解和参数签名 模块：functiontools.wraps**<br/>LINK:[装饰器文档](https://blog.csdn.net/HHG20171226/article/details/102658746)

In [47]:
from functools import wraps
def repeat(num):
    def my_decorator(func):
        @wraps(func)  #将原函数的元信息，拷贝到对应的装饰器函数里
        def wrapper(*args,**kwargs):
            for i in range(num):
                print("wrapper of decorator")
                func(*args,**kwargs)
        return wrapper
    return my_decorator
@repeat(3)  #相当于greet=repeat(4)(greet)  
def greet(message):
    print(message)
greet("hello,world")
print(greet.__name__) #元信息表示，greet作用域为greet

wrapper of decorator
hello,world
wrapper of decorator
hello,world
wrapper of decorator
hello,world
greet


### 类装饰器
**类装饰器主要依赖于函数__ call __(),每次调用都会进行重载调用**

In [73]:
class Count:
    def __init__(self,func):
        self.func=func
        self.num_calls=0
    def __call__(self,*args,**kwargs):
        self.num_calls+=1
        print(f"num of calls is:{self.num_calls}")
        return self.func(*args,**kwargs)
@Count    #example=Count(example)
def example(*args,**kwargs):
    print(*args,**kwargs)
example("hello,world!")
example("hello,world!")



num of calls is:1
hello,world!
num of calls is:2
hello,world!


### 装饰器的嵌套
python支持多个装饰器<br/>
比如：<br/>@decorator1<br/>
@decorator2<br/>
@decorator3<br/>
def func():<br/>
    ...<br/>
执行顺序从里到外<br/>
等同于decorator1(decorator2(decorator3(func)))

In [90]:
from functools import wraps
def my_decorator1(func):
    @wraps(func)
    def wrapper(*args,**kwargs):
        print("execute decorator1")
        func(*args,**kwargs)
    return wrapper
def my_decorator2(func):
    @wraps(func)
    def wrapper(*args,**kwargs):
        print("execute decorator2")
        func(*args,**kwargs)
    return wrapper
@my_decorator1
@my_decorator2
def greet(*args,**kwargs):
    print(*args,**kwargs)
greet("hello,world")
print(greet.__name__)

execute decorator1
execute decorator2
hello,world
greet


## 装饰器用法实例

### 身份认证(为某些功能服务）
需求：<br/>一些网站，你不登录也可以浏览内容，但如果你想要发布文章或留言，在点击发布时，服务器端便会查询你是否登录。如果没有登录，就不允许这项操作等等。<br/>实现：<br/>定义了装饰器 authenticate；而函数 post_comment()，则表示发表用户对某篇文章的评论。每次调用这个函数前，都会先检查用户是否处于登录状态，如果是登录状态，则允许这项操作；如果没有登录，则不允许。


In [ ]:
import functools
 
def authenticate(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        request = args[0]
        if check_user_logged_in(request): # 如果用户处于登录状态
            return func(*args, **kwargs) # 执行函数 post_comment() 
        else:
            raise Exception('Authentication failed')
    return wrapper
    
@authenticate
def post_comment(request, ...)
    ....

### 输入合理性检查
需求：<br/>在大型公司的机器学习框架中，我们调用机器集群进行模型训练前，往往会用装饰器对其输入（往往是很长的 json 文件）进行合理性检查。这样就可以大大避免，输入不正确对机器造成的巨大开销。<br/>实现：<br/>

In [ ]:
import functools
 
def validation_check(input):
    @functools.wraps(func)
    def wrapper(*args, **kwargs): 
        ... # 检查输入是否合法
    
@validation_check
def neural_network_training(param1, param2, ...):
    ...

### 缓存装饰器
**LRU cache，在 Python 中的表示形式是@lru_cache。<br/>@lru_cache会缓存进程中的函数参数和结果，当缓存满了以后，会删除 least recenly used 的数据。**<br/>正确使用缓存装饰器，往往能极大地提高程序运行效率。<br/>例子：大型公司服务器端的代码中往往存在很多关于设备的检查，比如你使用的设备是安卓还是 iPhone，版本号是多少。这其中的一个原因，就是一些新的 feature，往往只在某些特定的手机系统或版本上才有（比如 Android v200+）。这样一来，我们通常使用缓存装饰器，来包裹这些检查函数，避免其被反复调用，进而提高程序运行效率，比如写成下面这样：

In [ ]:
@lru_cache
def check(param1, param2, ...) # 检查用户设备类型，版本号等等
    ...